In [2]:
import pandas
import numpy as np
import nibabel as nib
from scipy.stats import zscore
import os

# Input

## HCP subject

One subject example taken from Human Connectome Project

- 119 rows: Representing 119 brain regions of interest (ROIs).

- 1200 columns: Representing time points or temporal samples.

In [4]:
sub1_input = "/data/etosato/RHOSTS/Input/subject1_left.txt"

In [5]:
np.loadtxt(sub1_input).shape

(1200, 119)

## Lorenzo's sample

### Load data

The atlas: Schaefer2018_100Parcels_17Networks_order_FSLMNI152_2mm.nii.gz

In [52]:
fmri_img = nib.load("/data/etosato/raw_data/HCP_rsfMRI/134829.nii.gz")      # NIfTI object
atlas_img = nib.load("/Preprocessing/atlases/cortex_100.nii.gz")
sub_atlas = nib.load("/Preprocessing/atlases/subcortex_16.nii")

In [53]:
fmri_data = fmri_img.get_fdata()    # array 4D
atlas_data = atlas_img.get_fdata()  # array 3D
sub_atlas_data = sub_atlas.get_fdata().astype(int)                 

In [54]:
print("Data shape: ", fmri_data.shape)
print("Atlas shape: ", atlas_data.shape)
print("Subcortical Atlas shape: ", sub_atlas_data.shape)

Data shape:  (91, 109, 91, 3600)
Atlas shape:  (91, 109, 91)
Subcortical Atlas shape:  (91, 109, 91)


In [55]:
np.unique(sub_atlas_data)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [19]:
print("Affine compatible?", np.allclose(fmri_img.affine, atlas_img.affine, atol=1e-3))

Affine compatible? True


### Mask averaging

#### Schaefer Parcellation

In [40]:
# Define the number of ROIs
n_rois = 100

# Define the number of timepoints (T)
T = fmri_data.shape[3]                          

# (T, n_rois)
ts = np.zeros((T, n_rois))                

# Loop over ROI
for roi in range(1, n_rois+1):
    # 1. Create the mask for the current ROI
    mask = ( roi == atlas_data)                              # (X, Y, Z) bool

    # 2. If there are no voxel for that roi, skip
    if not np.any(mask):
        continue

    # 3. Select the fMRI voxels belonging to this ROI --> (n_voxels, T)
    fmri_voxels = fmri_data[mask, :]

    # 4. Average across voxels to get a single time series --> (T,)
    roi_ts = fmri_voxels.mean(axis = 0)

    # 5. Store ROI time series as the correct column (roi - 1)
    ts[:, roi - 1] = roi_ts

In [41]:
fmri_voxels.shape

(863, 3600)

In [44]:
ts.shape

(3600, 100)

#### Subcortical

In [56]:
# Define the number of ROIs for the subcortical
n_rois_sub = 16
T = fmri_data.shape[3]
ts_sub = np.zeros((T, n_rois_sub))                

# Loop over ROI
for roi_sub in range(1, n_rois_sub+1):
    mask_sub = ( roi_sub == sub_atlas_data)                              

    if not np.any(mask_sub):
        continue

    fmri_voxels_sub = fmri_data[mask_sub, :]

    roi_ts_sub = fmri_voxels_sub.mean(axis = 0)

    ts_sub[:, roi_sub - 1] = roi_ts_sub

In [57]:
fmri_voxels_sub.shape

(435, 3600)

In [58]:
ts_sub.shape

(3600, 16)

#### Concatenation

In [59]:
ts_all = np.concatenate([ts, ts_sub], axis=1)  

### Z-score

In [62]:
ts_z = zscore(ts_all, axis=0)

print("Final shape (T x ROI):", ts_z.shape)

Final shape (T x ROI): (3600, 116)


## Txt

In [ ]:
out_path = f"{s_id}_ts_zscore.txt"
np.savetxt(out_path, ts_z, fmt="%.6f")

print("Saved:", out_path)

# Things

In [2]:
import h5py

src_hd5 = "/Output/sub1_left_weighted.hd5"  # large file
dst_hd5 = "/Output_temp/sub1_output_proj_small.hd5"  # reduced file

with h5py.File(src_hd5, "r") as src, h5py.File(dst_hd5, "w") as dst:
    # take the first five datasets in sorted order
    keys = sorted(src.keys(), key=lambda x: int(x))[:5]

    for k in keys:
        data = src[k][:]
        dst.create_dataset(k, data=data)

print("Created reduced HDF5 with datasets:", keys)


Created reduced HDF5 with datasets: ['0', '1', '2', '3', '4']
